### Imports

In [9]:
import numpy as np
import pandas as pd
from datetime import datetime
import time
import json
import requests
import hashlib
import urllib3
from urllib.parse import quote_plus
import hmac
import logging
from Config import *

### Login data

In [ ]:
api_key=api_key
api_secret=api_secret

### Log config

In [ ]:
logging.basicConfig(filename='info.log',
                    filemode="w",
                    level=logging.INFO,
                    format="{asctime} {levelname:<8} {message}",
                    style='{')

### Used functions

In [ ]:
def lot(symbol, df, api_key, api_secret):
    # Recalculate wallet ballace from given symbol to USD
    while True: 
        try:
            coin = symbol.replace("USD","")
            f = getCoin(coin, api_key, api_secret)             # Wallet balace

            rate = df["close"].iloc[-1]                        # rate from calcualtion taken from last close data
            return int(f*rate)
        except:
            logging.error('Bad connection at the getCoin method, trying again in 0.5 sec')
            time.sleep(0.5)
            pass

def getMarketData(symbol):
    # Compossited funtion: 
    # 1) Loads data from Bybit
    # 2) Calculates all nessesary atributes (SSL, HA, 6-Hour HA)
    
    # Get data from bybit via HTML request    
    since = int(time.time() - 129600)                          # 36 hours in seconds
    now = time.time()
    df = []
    while since < now:
        data = getData(symbol, since)
        df.append(data)
        since = since + 200*60                                 # interval is in min and timestamp in sec - thus *60 is needed to iterate by correct number
    
    
    df = pd.concat(df, ignore_index=True)
    
    df["open"] = pd.to_numeric(df["open"])
    df["close"] = pd.to_numeric(df["close"])
    df["high"] = pd.to_numeric(df["high"])
    df["low"] = pd.to_numeric(df["low"])
    
    # Compute Heiken-Ashi n SSL
    trend_buffer = pd.DataFrame(columns=["open", "close", "high", "low", "HA6_open", "HA6_close", "HA6_high", "HA6_low"])
    df['HA_close']=(df['open'] + df['high'] + df['low'] + df['close'])/4

    initial=1900                                               # for SSL and HA calculation it is not nessesary to iterate the whole dataset, just last 260 rows
    for i in range(initial,len(df)):
    #HA Open
        if i == initial:
            df.at[i, 'HA_open'] = ((df.at[i, 'open'] + df.at[i, 'close']) / 2) 
        else:
            df.at[i, 'HA_open'] = ((df.at[i-1, 'HA_open'] + df.at[i-1, 'HA_close']) / 2)
    
    df['HA_high'] = df[['HA_open','HA_close','high']].max(axis=1)
    df['HA_low'] = df[['HA_open','HA_close','low']].min(axis=1)
    df["HA_difference"] = df["HA_close"] - df["HA_open"]
    df["HA_relative"] = 100*df["HA_difference"]/df["open"]
    
    SSL = SSL
    Hlv0 = 1
    Hlv1 = 1
    df["SMA_high"] = df["HA_high"].ewm(span=SSL).mean()
    df["SMA_low"] = df["HA_low"].ewm(span=SSL).mean()

    
    for i in range(initial,len(df)):
                           
        if df.at[i, "close"] > df.at[i,"SMA_high"]:
            Hlv0 = 1
        elif df.at[i, "close"] < df.at[i,"SMA_low"]:
            Hlv0 = -1
        else:
            Hlv0 = Hlv1
        if Hlv0 < 0:
            df.at[i, "SSL_down"] = df.at[i, "SMA_high"]
            df.at[i, "SSL_up"] = df.at[i, "SMA_low"]
            Hlv1 = Hlv0        
        else:
            df.at[i, "SSL_down"] = df.at[i, "SMA_low"]
            df.at[i, "SSL_up"] = df.at[i, "SMA_high"]
            Hlv1 = Hlv0
        if  df.at[i, "SSL_up"] > df.at[i, "SSL_down"]:
            df.at[i, "signal"] = 1.0
        else:
            df.at[i, "signal"] = 0.0
    
    #HA long interval
    for j in range(0, 6):
        trend_buffer.at[j, "open"] = df["open"].iat[0+j*360]
        trend_buffer.at[j, "close"] = df["close"].iat[358+j*360]

        trend_buffer.at[j, "high"] = df["high"].iloc[(0+j*360):(j+1)*360-1].max()
        trend_buffer.at[j, "low"] = df["low"].iloc[(0+j*360):(j+1)*360-1].min()
    for j in range(0, 6):
        trend_buffer['HA6_close']=(trend_buffer['open'] + trend_buffer['high'] + trend_buffer['low'] + trend_buffer['close'])/4
        if j == 0:
            trend_buffer.at[j, 'HA6_open'] = (trend_buffer.at[j, 'open'] + trend_buffer.at[j, 'close']) / 2
        else:
            trend_buffer.at[j, 'HA6_open'] = (trend_buffer.at[j-1, 'HA6_open'] + trend_buffer.at[j-1, 'HA6_close']) / 2
        trend_buffer['HA6_high'] = trend_buffer[['HA6_open','HA6_close','high']].max(axis=1)
        trend_buffer['HA6_low'] = trend_buffer[['HA6_open','HA6_close','low']].min(axis=1)

    tren_str = trend_buffer.at[5, "HA6_close"] - trend_buffer.at[5 ,"HA6_open"]
    tren_str = tren_str/trend_buffer.at[5, "HA6_open"]*100
    
    df = df.fillna("0.0")                                      # Due to skipping first 1900 rows with SSL and HA it is nessesary to change na values to 0                   
    
    return df, tren_str

def isBreak(data, limit_break):
    df = data
    if (
        (df["HA_relative"].iat[-2] > limit_break) and 
        (df["HA_close"].iat[-1] > df["HA_close"].iat[-2]) and 
        (df["HA_high"].iat[-1] > df["HA_high"].iat[-2])):
            return "Buy"

    elif (
        (df["HA_relative"].iat[-2] < -1*limit_break) and 
        (df["HA_close"].iat[-1] < df["HA_close"].iat[-2]) and 
        (df["HA_low"].iat[-1] < df["HA_low"].iat[-2])):
            return "Sell"
    else:
            return "None"

def isTrend(limit_trend, trend_strength):
    if  trend_strength > limit_trend:
        return "Buy"
    elif  trend_strength < -1*limit_trend:
        return "Sell"
    else:
        return "None"
    
def getPosition(symbol, api_key, api_secret):    
    # Gets info about current position state
    # Side - Buy, Sell, None
    # Size - 0 if no position, qty for buy/sell
    params = { 
        "symbol": symbol,
        "api_key": api_key,
        "timestamp": round(time.time() * 1000),
        "recv_window": 10000,
    }
    param_str = ""
    for key in sorted(params.keys()):
        v = params[key]
        if isinstance(params[key], bool):
            if params[key]:
                v = "true"
            else:
                v = "false"
        param_str += f"{key}={v}&"
    param_str = param_str[:-1]

    hash = hmac.new(bytes(api_secret, "utf-8"), param_str.encode("utf-8"),
                    hashlib.sha256)
    signature = hash.hexdigest()
    sign_real = {
        "sign": signature
    }

    param_str = quote_plus(param_str, safe="=&")
    full_param_str = f"{param_str}&sign={sign_real['sign']}"

    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    body = None
    url = "https://api-testnet.bybit.com/v2/private/position/list"
    urllib3.disable_warnings()
    s = requests.session()
    s.keep_alive = False

    method = "GET"
    response = requests.request(method, f"{url}?{full_param_str}", headers=headers, verify=False)
    l = json.loads(response.text)["result"]
    df = pd.DataFrame(l, index=[0])
    side = str(df.iat[0,6])
    size = int(df.iat[0,7])
    leverage = int(df.iat[0,12])
    return side, size, leverage

def getCoin(coin, api_key, api_secret):    
    # Gets wallet ballance for given coin
    # USD from symbol has to be dropped
    params = { 
        "coin": coin,
        "api_key": api_key,
        "timestamp": round(time.time() * 1000),
        "recv_window": 10000,
    }
    param_str = ""
    for key in sorted(params.keys()):
        v = params[key]
        if isinstance(params[key], bool):
            if params[key]:
                v = "true"
            else:
                v = "false"
        param_str += f"{key}={v}&"
    param_str = param_str[:-1]

    hash = hmac.new(bytes(api_secret, "utf-8"), param_str.encode("utf-8"),
                    hashlib.sha256)
    signature = hash.hexdigest()
    sign_real = {
        "sign": signature
    }

    param_str = quote_plus(param_str, safe="=&")
    full_param_str = f"{param_str}&sign={sign_real['sign']}"

    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    body = None
    url = "https://api-testnet.bybit.com/v2/private/wallet/balance"
    urllib3.disable_warnings()
    s = requests.session()
    s.keep_alive = False

    method = "GET"
    response = requests.request(method, f"{url}?{full_param_str}", headers=headers, verify=False)
    l = json.loads(response.text)["result"][coin]
    return l["equity"]

def placeOrder(side, symbol, qty, api_key, api_secret):
    params = { 
        "side": side,
        "symbol": symbol,
        "qty": qty,
        "order_type": "Market",
        "time_in_force": "GoodTillCancel",
        "api_key": api_key,
        "timestamp": round(time.time() * 1000),
        "recv_window": 10000,
    }
    param_str = ""
    for key in sorted(params.keys()):
        v = params[key]
        if isinstance(params[key], bool):
            if params[key]:
                v = "true"
            else:
                v = "false"
        param_str += f"{key}={v}&"
    param_str = param_str[:-1]

    hash = hmac.new(bytes(api_secret, "utf-8"), param_str.encode("utf-8"),
                    hashlib.sha256)
    signature = hash.hexdigest()
    sign_real = {
        "sign": signature
    }

    param_str = quote_plus(param_str, safe="=&")
    full_param_str = f"{param_str}&sign={sign_real['sign']}"

    headers = {"Content-Type": "application/json"}
    body = dict(params, **sign_real)
    url = "https://api-testnet.bybit.com//v2/private/order/create"
    urllib3.disable_warnings()
    s = requests.session()
    s.keep_alive = False

    method = "POST"
    response = requests.request(method, url, data=json.dumps(body), headers=headers, verify=False)
    return None

def getData(symbol, since):    
    params = { 
        "symbol": symbol,
        "from": since,
        "interval": "1",
        "timestamp": round(time.time() * 1000),
        "recv_window": 10000,
    }
    param_str = ""
    for key in sorted(params.keys()):
        v = params[key]
        if isinstance(params[key], bool):
            if params[key]:
                v = "true"
            else:
                v = "false"
        param_str += f"{key}={v}&"
    param_str = param_str[:-1]

    hash = hmac.new(bytes(api_secret, "utf-8"), param_str.encode("utf-8"),
                    hashlib.sha256)
    signature = hash.hexdigest()
    sign_real = {
        "sign": signature
    }

    param_str = quote_plus(param_str, safe="=&")
    full_param_str = f"{param_str}&sign={sign_real['sign']}"

    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    body = None
    url = "https://api-testnet.bybit.com/v2/public/kline/list"
    urllib3.disable_warnings()
    s = requests.session()
    s.keep_alive = False

    method = "GET"
    response = requests.request(method, f"{url}?{full_param_str}", headers=headers, verify=False)
    l = json.loads(response.text)["result"]
    l = pd.DataFrame(l)
    return l

def setLeverage(symbol, leverage, api_key, api_secret):        
    params = { 
        "symbol": symbol,
        "leverage": leverage,
        "api_key": api_key,
        "timestamp": round(time.time() * 1000),
        "recv_window": 10000,
    }
    param_str = ""
    for key in sorted(params.keys()):
        v = params[key]
        if isinstance(params[key], bool):
            if params[key]:
                v = "true"
            else:
                v = "false"
        param_str += f"{key}={v}&"
    param_str = param_str[:-1]

    hash = hmac.new(bytes(api_secret, "utf-8"), param_str.encode("utf-8"),
                    hashlib.sha256)
    signature = hash.hexdigest()
    sign_real = {
        "sign": signature
    }

    param_str = quote_plus(param_str, safe="=&")
    full_param_str = f"{param_str}&sign={sign_real['sign']}"

    headers = {"Content-Type": "application/json"}
    body = dict(params, **sign_real)
    url = "https://api-testnet.bybit.com//v2/private/position/leverage/save"
    urllib3.disable_warnings()
    s = requests.session()
    s.keep_alive = False

    method = "POST"
    response = requests.request(method, url, data=json.dumps(body), headers=headers, verify=False)
    return None

def calcLeverage(data):
    # calculate leverage for given market movement based of liquidation%
    # WIP different symbols has different percentages BTC only for now
    
    last_agression = abs(data["HA_relative"].iat[-1])
    if last_agression > 16.32:
        return 1
    elif last_agression > 8.68:
        return 5
    elif last_agression > 4.31:
        return 10
    elif last_agression > 2.76:
        return 20
    elif last_agression > 1.96:
        return 30
    elif last_agression > 1.48:
        return 40
    else:
        return 50

### Trade function

In [ ]:
def doTrade(data, symbol, trend_strength, api_key, api_secret):
    risk = risk
    limit_break = limit_break
    limit_trend = limit_trend 
    #Get last position
    while True:
        try:
            position, qty, leverage = getPosition(symbol, api_key, api_secret)
            break
        except:
            logging.error('Bad connection at the getPosition method, trying again in 0.5 sec')
            time.sleep(0.5)
            pass
     
    #Get out
    if position != "None":
        df = data
        if position == "Buy":
            if ((df["signal"].iat[-1] - df["signal"].iat[-2]) == -1):
                while True:
                    try:
                        placeOrder(side="Sell", symbol=symbol, qty=qty, api_key=api_key, api_secret=api_secret)
                        logging.info("Close Buy Trade")
                        break
                    except:
                        logging.error("Connect issue at Close buy position, trying again in 0.5 sec")
                        time.sleep(0.5)
                        pass
            
        elif position == "Sell":
            if ((df["signal"].iat[-1] - df["signal"].iat[-2]) == 1):
                while True:
                    try:
                        placeOrder(side="Buy", symbol=symbol, qty=qty, api_key=api_key, api_secret=api_secret)
                        logging.info("Close Sell Trade")
                        break
                    except:
                        logging.error("Connect issue at Close sell position, trying again in 0.5 sec")
                        time.sleep(0.5)
                        pass
    #Get in            
    else:
        df = data
        if (isBreak(df, limit_break) =="Buy") and (isTrend(limit_trend, trend_strength)=="Buy"):
            newLeverage = calcLeverage(df)
            entry = lot(symbol, df, api_key, api_secret)*risk*newLeverage     
            if leverage != newLeverage:                        # Only try to set the new leverage when it is different from current leverage
                while True:
                    try:
                        setLeverage(symbol=symbol, leverage = newLeverage, api_key = api_key, api_secret = api_secret)
                        logging.info("Setting leverage to" + str(newLeverage))
                        break
                    except:
                        logging.error("Issue while setting the leverage, trying in 0.5 sec")
                        time.sleep(0.5)
                        pass
             
            while True:
                try:           
                    placeOrder(side="Buy", symbol=symbol, qty=entry, api_key=api_key, api_secret=api_secret)
                    logging.info("Open Buy Trade")
                    break
                except:
                    logging.error("Connect issue at Open buy position, trying again in 0.5 sec")
                    time.sleep(0.5)
                    pass
         
        elif (isBreak(df, limit_break) =="Sell") and (isTrend(limit_trend, trend_strength)=="Sell"):
            newLeverage = calcLeverage(df)
            entry = lot(symbol, df, api_key, api_secret)*risk*newLeverage    
            if leverage != newLeverage:  
                while True:
                    try:                        
                        setLeverage(symbol=symbol, leverage = newLeverage, api_key = api_key, api_secret = api_secret)
                        logging.info("Setting leverage to" + str(newLeverage))
                        break
                    except:
                        logging.error("Issue while setting the leverage, trying in 0.5 sec")
                        time.sleep(0.5)
                        pass
                
            while True:
                try:                        
                    placeOrder(side="Sell", symbol=symbol, qty=entry, api_key=api_key, api_secret=api_secret)
                    logging.info("Close Buy Trade")
                    break
                except:
                    logging.error("Connect issue at Open sell position, trying again in 0.5 sec")
                    time.sleep(0.5)
                    pass
    return None

### Main script loop

In [ ]:
symbol = "BTCUSD"
while True:
    try:
        data, trend_strength = getMarketData(symbol)
        doTrade(data, symbol, trend_strength, api_key, api_secret)
        time.sleep((60 - time.time() % 60)+1)
    except Exception as e:
        logging.critical(e)
        break